# Imports and definitions

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import pandas as pd
from sklearn import svm
from sklearn.datasets import make_blobs
from sklearn.linear_model import SGDClassifier

In [15]:

def batch_normalize(X, sigma=None):
    d, n = X.shape
    # print(X)
    mu = np.mean(X, axis=1, keepdims=True)
    Xbar = X - mu
    sigma = np.sqrt((np.diag(Xbar @ Xbar.T) + 1e-10) / n).reshape((d, 1)) if sigma is None else sigma
    # print(Xbar)
    # print(sigma)
    return Xbar / sigma


def plot_blobs(X, y, title=None):
    y_unique = np.unique(y)
    colors = cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        # this_sw = sw_train[y_train == this_y]
        plt.scatter(
            this_X[:, 0],
            this_X[:, 1],
            # s=this_sw * 50,
            c=color[np.newaxis, :],
            alpha=0.5,
            edgecolor="k",
            label="Class %s" % this_y,
        )
    plt.legend(loc="best")
    plt.title(title if title is not None else 'Data')

    plt.show()

def plot_sgd_classifier(X, y, C, contour=False, comparison_clf=None):
    """
    Returns an unregularized logistic regression classifier with step size 0.01 and 10000 iterations
    Also plots the classifier and data
    """
    clf = SGDClassifier(loss='log', alpha=0, learning_rate='constant', eta0=0.001, max_iter=10000, fit_intercept=False)
    clf.fit(X, y)
    decision_function = clf.decision_function(X)
    # we can also calculate the decision function manually
    # decision_function = np.dot(X, clf.coef_[0]) + clf.intercept_[0]
    # The support vectors are the samples that lie within the margin
    # boundaries, whose size is conventionally constrained to 1

    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    xx, yy = np.meshgrid(
        np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50)
    )
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    if comparison_clf is not None:
        Z_comparison = comparison_clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        Z_comparison = Z_comparison.reshape(xx.shape)
        print(clf.coef_)
        print(comparison_clf.coef_)

        print(np.linalg.norm(Z_comparison - Z))
        

    if contour:
        plt.contour(
            xx,
            yy,
            Z,
            colors="k",
            levels=[-1, 0, 1],
            alpha=0.5,
            linestyles=["--", "-", "--"],
        )
    else:
        plt.contour(
            xx,
            yy,
            Z,
            colors="k",
            levels=[0],
            alpha=0.5,
            linestyles=["-"],
        )

        if comparison_clf is not None:
            plt.contourf(
                xx,
                yy,
                Z_comparison,
                colors=["b", "m", "r"],
                levels=[-1, 0, 1],
                alpha=0.5,
            )
    return clf

def plot_max_margin_classifier(X, y, C, contour=False):
    clf = svm.SVC(kernel="linear", C=C)
    clf.fit(X, y)
    decision_function = clf.decision_function(X)
    # we can also calculate the decision function manually
    # decision_function = np.dot(X, clf.coef_[0]) + clf.intercept_[0]
    # The support vectors are the samples that lie within the margin
    # boundaries, whose size is conventionally constrained to 1
    support_vector_indices = np.where(np.abs(decision_function) <= 1 + 1e-15)[0]
    support_vectors = X[support_vector_indices]

    plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    xx, yy = np.meshgrid(
        np.linspace(xlim[0], xlim[1], 50), np.linspace(ylim[0], ylim[1], 50)
    )
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    if contour:
        plt.contour(
            xx,
            yy,
            Z,
            colors="k",
            levels=[-1, 0, 1],
            alpha=0.5,
            linestyles=["--", "-", "--"],
        )
    else:
        plt.contour(
            xx,
            yy,
            Z,
            colors="k",
            levels=[0],
            alpha=0.5,
            linestyles=["-"],
        )
    # plt.scatter(
    #     support_vectors[:, 0],
    #     support_vectors[:, 1],
    #     s=100,
    #     linewidth=1,
    #     facecolors="none",
    #     edgecolors="k",
    # )

# Random reshuffle optimum computation

In [ ]:
import itertools

In [ ]:
from contextlib import nullcontext
# maybe you can optimize this by permuting indices rather than generating permutatiuons of the iterable itself
def compute_rr_minimizer(X, Y, m=2, sigma=None):
    first_moment = 0
    second_moment = 0

    n = X.shape[1]
    ss_optima = []
    for pi in itertools.permutations(list(range(n))):
        list_Xtilde = []
        for i in range(n//m):
            Xtilde = batch_normalize(X[:, pi[m*i:m*i+m]], sigma=sigma)
            list_Xtilde.append(Xtilde)

        X_pi = np.hstack(list_Xtilde)
        ss_optima.append(compute_ss_minimizer(X_pi, Y[:, pi]))
            # print(X_pi)
        first_moment += Y[:, pi] @ X_pi.T
        second_moment += X_pi @ X_pi.T
    print(first_moment, second_moment)
    print(ss_optima, '\naverage optimum:\n', np.mean(ss_optima, axis=0))
    return first_moment @ np.linalg.inv(second_moment), batch_normalize(X) 

def compute_approx_rr_minimizer(X, Y, m=2, k=1000, sigma=None):
    first_moment = 0
    second_moment = 0

    n = X.shape[1]
    list_n = list(range(n))
    ss_optima = []
    for i in range(k):
        pi = np.random.permutation(list_n)
        list_Xtilde = []
        for i in range(n//m):
            Xtilde = batch_normalize(X[:, pi[m*i:m*i+m]], sigma=sigma)
            list_Xtilde.append(Xtilde)

        X_pi = np.hstack(list_Xtilde)
        ss_optima.append(compute_ss_minimizer(X_pi, Y[:, pi]))
            # print(X_pi)
        first_moment += Y[:, pi] @ X_pi.T
        # print(X_pi @ X_pi.T)
        second_moment += X_pi @ X_pi.T
    # print(first_moment, second_moment)
    # print(ss_optima, '\naverage optimum:\n', np.mean(ss_optima, axis=0))
    return first_moment @ np.linalg.inv(second_moment), batch_normalize(X) 

def compute_fb_minimizer(X, Y):
    X_fb = batch_normalize(X)
    return compute_ss_minimizer(X, Y)

def compute_ss_minimizer(X_pi, Y):
    return np.linalg.lstsq(X_pi.T, Y.T)[0].T

def compute_ols_minimizer(X, Y):
    return np.linalg.lstsq(X.T, Y.T)[0].T

In [ ]:
X, Y = np.random.random((2, 9)), np.random.random((1, 9))

W_RR, X_RR = compute_approx_rr_minimizer(X, Y, m=3, k=10)
W_OLS = compute_ols_minimizer(X, Y)
print(f'approx W_RR: {W_RR}, approx Y_RR: {W_RR @ X_RR}')
print(f'W_OLS: {W_OLS}, Y_OLS: {W_OLS @ X}')

[[8.99999999]]
[[8.99999999]]
[[8.99999999]]
[[8.99999999]]
[[8.99999999]]
[[8.99999999]]
[[8.99999999]]
[[8.99999997]]
[[8.99999999]]
[[8.99999999]]
[[-13.51435866]] [[89.99999988]]
[array([[-0.15129633]]), array([[-0.17814268]]), array([[-0.21201168]]), array([[-0.09565751]]), array([[-0.14830785]]), array([[-0.16038982]]), array([[-0.12603961]]), array([[-0.1381756]]), array([[-0.17466516]]), array([[-0.11690917]])] 
average optimum:
 [[-0.15015954]]
approx W_RR: [[-0.15015954]], approx Y_RR: [[-0.26368615  0.18418914 -0.20293873  0.03085798  0.18541331 -0.05598088
  -0.00274884  0.13997073 -0.01507657]]
W_OLS: [[0.6152851]], Y_OLS: [[0.60771961 0.16689507 0.54792851 0.3178124  0.16569018 0.40328418
  0.35089015 0.21041737 0.36302381]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [ ]:
n = 9
m = 3
X = np.random.randn(2, n)
Y = np.array([[1, 2]]) @ X + 0.01 * np.random.randn(1, n)
pi = np.random.permutation(n).tolist()
pi_inv = [0] * n
for i, k in enumerate(pi):
    pi_inv[k] = i


list_Xtilde = []
for i in range(n//m):
    Xtilde = batch_normalize(X[:, pi[m*i:m*i+m]])
    list_Xtilde.append(Xtilde)

X_pi = np.hstack(list_Xtilde)
W_pi = compute_ss_minimizer(X_pi, Y[:, pi])
X_fb = batch_normalize(X)
W_fb = compute_fb_minimizer(X, Y)

W_RR, X_RR = compute_approx_rr_minimizer(X, Y, m=m, k=10000)

print(f'W_pi: {W_pi}, Y_pi = {(W_pi @ X_pi)[:, pi_inv]}')
print(f'W_fb: {W_fb}, Y_fb = {W_fb @ X_fb}')
print(f'approx W_RR: {W_RR}, approx Y_RR: {W_RR @ X_RR}')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


W_pi: [[0.60538439 0.47435554]], Y_pi = [[-1.0381235   1.46510437  0.42359092 -0.13558303 -0.28800789  0.0217243
   0.50628099 -0.42698087 -0.52800529]]
W_fb: [[1.00006928 2.00617451]], Y_fb = [[-4.76334352  0.72487557  1.82990294  1.36166495  0.57368544  0.06054656
   2.71301181 -4.09290888  1.59256513]]
approx W_RR: [[0.70023744 0.58607014]], approx Y_RR: [[-1.93172041  0.11664357  0.66137626  0.55893561 -0.41503733  0.66997119
   1.25844332 -1.56923683  0.65062462]]


In [ ]:
n = 8
m = 4
X = np.random.randn(2, n)
Y = np.array([[1, 2]]) @ X + 0.01 * np.random.randn(1, n)
pi = np.random.permutation(n).tolist()
pi_inv = [0] * n
for i, k in enumerate(pi):
    pi_inv[k] = i


list_Xtilde = []
for i in range(n//m):
    Xtilde = batch_normalize(X[:, pi[m*i:m*i+m]], sigma=1)
    list_Xtilde.append(Xtilde)

X_pi = np.hstack(list_Xtilde)
W_pi = compute_ss_minimizer(X_pi, Y[:, pi])
X_fb = batch_normalize(X, sigma=1)
W_fb = compute_fb_minimizer(X, Y)

W_RR, X_RR = compute_rr_minimizer(X, Y, m=m, sigma=1)

print(f'W_pi: {W_pi}, Y_pi = {(W_pi @ X_pi)[:, pi_inv]}')
print(f'W_fb: {W_fb}, Y_fb = {W_fb @ X_fb}')
print(f'approx W_RR: {W_RR}, approx Y_RR: {W_RR @ X_RR}')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


[[360609.0858468  711295.82248715]] [[419593.32450632 -29944.19246992]
 [-29944.19246992 371141.2682772 ]]
[array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00069746, 1.99474913]]), array([[1.00192444, 1.99679627]]), array([[1.0019244

In [ ]:
n = 1000
m = 4
X = np.random.randn(2, n)
Y = np.array([[1, 2]]) @ X + 0.01 * np.random.randn(1, n)
pi = np.random.permutation(n).tolist()
pi_inv = [0] * n
for i, k in enumerate(pi):
    pi_inv[k] = i


list_Xtilde = []
for i in range(n//m):
    Xtilde = batch_normalize(X[:, pi[m*i:m*i+m]], sigma=1)
    list_Xtilde.append(Xtilde)

X_pi = np.hstack(list_Xtilde)
W_pi = compute_ss_minimizer(X_pi, Y[:, pi])
X_fb = batch_normalize(X, sigma=1)
W_fb = compute_fb_minimizer(X, Y)

W_RR, X_RR = compute_approx_rr_minimizer(X, Y, m=m, k=1000, sigma=1)

print(f'W_pi: {W_pi}, Y_pi = {(W_pi @ X_pi)[:, pi_inv]}')
print(f'W_fb: {W_fb}, Y_fb = {W_fb @ X_fb}')
print(f'approx W_RR: {W_RR}, approx Y_RR: {W_RR @ X_RR}')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


W_pi: [[0.99982769 1.99979644]], Y_pi = [[ 2.12141350e+00  1.30625529e+00  2.97697774e+00  1.93698615e+00
   2.72268118e+00 -1.11527902e+00 -2.05256860e-02  6.48540387e-01
   1.76474282e+00  1.61106563e+00 -6.88117550e-01  1.84218638e+00
   3.73520109e+00 -4.27177091e+00  7.72534522e-01 -1.54133073e+00
   3.25923293e-01  2.91183367e+00 -9.87443210e-01 -6.99787173e-01
   7.00586634e-01 -7.32750026e-01 -1.91875385e+00 -6.26992629e-01
  -1.82403977e-01 -2.29324955e+00 -2.58773002e+00 -1.56611742e+00
   7.00712406e-01 -7.34001586e-01 -9.51859325e-01  2.45996386e-01
   1.90207797e+00  1.57924878e+00 -5.21883523e-01 -3.28464017e-01
  -2.99183899e+00  1.58575918e+00  2.91122887e-01  4.97918619e-01
   1.87854275e+00  9.45133183e-01 -1.59316092e+00  1.19542220e+00
   5.17504366e-01  6.15237028e-01  1.62485156e+00  2.63607032e+00
  -9.75968250e-01 -1.36104949e+00  1.05062252e+00  5.95528260e-01
  -1.80912153e+00 -6.27276089e-01  2.35239411e+00 -2.10391142e+00
   1.22020679e+00 -6.23749812e-01  9

In [ ]:
X, Y = np.random.random((1, 8)), np.random.random((1, 8))

W_RR, X_RR = compute_rr_minimizer(X, Y, m=4)
W_OLS = compute_ols_minimizer(X, Y)
print(f'W_RR: {W_RR}, Y_RR: {W_RR @ X_RR}')
print(f'W_OLS: {W_OLS}, Y_OLS: {W_OLS @ X}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


[[-3428.26746223]] [[322559.99931597]]
[array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.06960908]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.00577285]]), array([[0.0057728

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
